In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col, log

import mlflow
import mlflow.spark

# %%
spark = SparkSession.builder \
    .appName("SECOP_MLflow") \
    .master("local[*]") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/13 16:13:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### **Configurar MLflow tracking server y experimento**

In [2]:
mlflow.set_tracking_uri("http://mlflow:5000")

In [3]:
experiment_name="secop_prediccion"
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='file:///opt/mlflow/mlruns/503782568921020891', creation_time=1770998094620, experiment_id='503782568921020891', last_update_time=1770998094620, lifecycle_stage='active', name='secop_prediccion', tags={}>

26/02/13 16:13:38 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:
df = spark.read.parquet("/opt/spark-data/raw/secop_ml_ready1.parquet")
df = df.withColumnRenamed("valor_del_contrato_log", "label") \
       .withColumnRenamed("features_pca", "features") \
       .filter(col("label").isNotNull())

# 🔥 MUY IMPORTANTE: eliminar ceros o negativos antes del log
# df = df.filter(col("label") > 0)

train, test = df.randomSplit([0.8, 0.2], seed=42)

print(f"Train: {train.count():,}")
print(f"Test: {test.count():,}")

# %%
evaluator_rmse = RegressionEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="rmse"
)

evaluator_mae = RegressionEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="mae"
)
evaluator_r2 = RegressionEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="r2"
)

Train: 80,141


Test: 19,859


### **Registrar experimento baseline con log_param/log_metric**

In [5]:
print("Experimento 1")
with mlflow.start_run(run_name="baseline_model"):
    # definamos parametros
    reg_param = 0.1
    elastic_param = 0.0
    max_iter = 100

    # Log de Parámetros
    mlflow.log_param("regParam", reg_param)
    mlflow.log_param("elasticParam", elastic_param)
    mlflow.log_param("maxIaram", max_iter)

    # Entrenar Modelo
    lr= LinearRegression(
        featuresCol="features",
        labelCol="label",
        regParam=reg_param,
        elasticNetParam=elastic_param,
        maxIter=max_iter
    )

    model = lr.fit(train)

    # Generar Predicciones
    predictions = model.transform(test)
    
    # Evaluar modelo
    rmse = evaluator_rmse.evaluate(predictions)
    mae = evaluator_mae.evaluate(predictions)
    r2 = evaluator_r2.evaluate(predictions)

    # Log de metricas
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("mae", mae)
    mlflow.log_metric("r2", r2)

    # Guardar modelo
    mlflow.spark.log_model(model, "model")

    

    print(f"✓ RMSE: ${rmse:,.2f}")
    print(f"✓ MAE: ${mae:,.2f}")
    print(f"✓ R²: {r2:.4f}")




Experimento 1


2026/02/13 16:13:58 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet

26/02/13 16:14:05 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBL

✓ RMSE: $1.39
✓ MAE: $0.72
✓ R²: 0.7143


### **Registrar multiples modelos (Ridge, Lasso, ElasticNet)**

In [6]:
print("REGISTRANDO MODELOS CON DIFERENTE REGULARIZACIÓN")

# Lista de configuraciones
experiments = [
    {"name": "ridge_l2_regression", "reg": 0.1, "elastic": 0.0, "type": "Ridge"},
    {"name": "lasso_l1_regression", "reg": 0.1, "elastic": 1.0, "type": "Lasso"},
    {"name": "elasticnet_l1_l2", "reg": 0.1, "elastic": 0.5, "type": "ElasticNet"},
]

# Evaluadores
evaluator_rmse = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
evaluator_mae = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="mae")
evaluator_r2 = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="r2")

for exp in experiments:

    print(f"\n=== EXPERIMENTO: {exp['type']} ===")

    with mlflow.start_run(run_name=exp["name"]):

        # -----------------------------
        # Log de parámetros
        # -----------------------------
        mlflow.log_param("regParam", exp["reg"])
        mlflow.log_param("elasticNetParam", exp["elastic"])
        mlflow.log_param("maxIter", 100)
        mlflow.log_param("model_type", exp["type"])

        # -----------------------------
        # Entrenar modelo
        # -----------------------------
        lr = LinearRegression(
            featuresCol="features",
            labelCol="label",
            regParam=exp["reg"],
            elasticNetParam=exp["elastic"],
            maxIter=100
        )

        model = lr.fit(train)

        # -----------------------------
        # Predicciones
        # -----------------------------
        predictions = model.transform(test)

        # -----------------------------
        # Métricas
        # -----------------------------
        rmse = evaluator_rmse.evaluate(predictions)
        mae = evaluator_mae.evaluate(predictions)
        r2 = evaluator_r2.evaluate(predictions)

        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("mae", mae)
        mlflow.log_metric("r2", r2)

        # -----------------------------
        # Guardar modelo
        # -----------------------------
        mlflow.spark.log_model(model, "model")

        print(f"✓ RMSE: ${rmse:,.2f}")
        print(f"✓ MAE:  ${mae:,.2f}")
        print(f"✓ R²:   {r2:.4f}")

print("\n" + "="*60)
print("✓ 3 MODELOS REGISTRADOS EN MLFLOW")
print("✓ Accede a MLflow UI: http://localhost:5000")
print("="*60)


REGISTRANDO MODELOS CON DIFERENTE REGULARIZACIÓN

=== EXPERIMENTO: Ridge ===


2026/02/13 16:19:01 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpftmiidzh/model, flavor: spark), fall back to return ['pyspark==3.5.0']. Set logging level to DEBUG to see the full traceback.


✓ RMSE: $1.39
✓ MAE:  $0.72
✓ R²:   0.7143

=== EXPERIMENTO: Lasso ===


2026/02/13 16:21:19 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmphae5jkex/model, flavor: spark), fall back to return ['pyspark==3.5.0']. Set logging level to DEBUG to see the full traceback.


✓ RMSE: $1.45
✓ MAE:  $0.76
✓ R²:   0.6896

=== EXPERIMENTO: ElasticNet ===


2026/02/13 16:23:35 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpl9svwhqr/model, flavor: spark), fall back to return ['pyspark==3.5.0']. Set logging level to DEBUG to see the full traceback.


✓ RMSE: $1.42
✓ MAE:  $0.74
✓ R²:   0.7025

✓ 3 MODELOS REGISTRADOS EN MLFLOW
✓ Accede a MLflow UI: http://localhost:5000


Registrar múltiples métricas permite evaluar el modelo desde diferentes perspectivas. RMSE penaliza fuertemente errores grandes, MAE mide el error promedio absoluto sin amplificar outliers y R² indica qué proporción de la varianza es explicada por el modelo. Un modelo puede tener buen RMSE pero bajo R², o viceversa. Registrar varias métricas permite tomar decisiones más informadas y comparar modelos con mayor profundidad.

### **Explorar y comparar runs en MLflow UI**

**¿Qué modelo tiene el menor RMSE?**  
El modelo Ridge (L2) presenta el menor RMSE dentro de los experimentos evaluados, aunque la diferencia frente a Lasso, ElasticNet y el modelo baseline es mínima y no representa una mejora significativa en términos prácticos.

**¿Hay correlación entre regularización y rendimiento?**  
No se observa una correlación clara entre el nivel o tipo de regularización y el rendimiento del modelo. Los valores de RMSE, MAE y R² son prácticamente iguales en los cuatro modelos, lo que indica que la regularización aplicada (regParam=0.1) no genera un impacto relevante en la capacidad predictiva bajo este conjunto de datos.

**¿Cómo podrías compartir estos resultados con tu equipo?**  
Los resultados pueden compartirse mediante la interfaz de MLflow UI, utilizando la opción de comparación de runs o el botón “Share” del experimento. También es posible exportar métricas, descargar artefactos registrados (modelos y reportes) o integrar MLflow con un repositorio compartido para que el equipo pueda reproducir y auditar los experimentos.

### **Agregar artefactos personalizados (reportes, graficos)**

In [7]:
import matplotlib.pyplot as plt
import pandas as pd
import tempfile

print("\n===  Modelo con Artefactos ===")

with mlflow.start_run(run_name="model_with_artifacts"):

    # Usamos los mejores hiperparámetros encontrados
    reg_param = 0.1
    elastic_param = 0.0  # Ridge
    max_iter = 100

    mlflow.log_param("regParam", reg_param)
    mlflow.log_param("elasticNetParam", elastic_param)
    mlflow.log_param("maxIter", max_iter)
    mlflow.log_param("model_type", "Ridge")

    # Entrenar modelo
    lr = LinearRegression(
        featuresCol="features",
        labelCol="label",
        regParam=reg_param,
        elasticNetParam=elastic_param,
        maxIter=max_iter
    )

    model = lr.fit(train)
    predictions = model.transform(test)

    # Evaluar métricas
    rmse = evaluator_rmse.evaluate(predictions)
    mae = evaluator_mae.evaluate(predictions)
    r2 = evaluator_r2.evaluate(predictions)

    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("mae", mae)
    mlflow.log_metric("r2", r2)

    # ==============================
    # 1️⃣ REPORTE EN TEXTO
    # ==============================

    report = f"""
    REPORTE DE MODELO
    ==================
    Modelo: Ridge Regression
    regParam: {reg_param}
    elasticNetParam: {elastic_param}
    maxIter: {max_iter}

    MÉTRICAS:
    RMSE: ${rmse:,.2f}
    MAE: ${mae:,.2f}
    R²: {r2:.4f}
    """

    mlflow.log_text(report, "model_report.txt")

    # ==============================
    # 2️⃣ GRÁFICO REAL VS PREDICHO
    # ==============================

    # Convertir pequeña muestra a pandas
    sample_pd = predictions.select("label", "prediction").limit(1000).toPandas()

    plt.figure(figsize=(6,6))
    plt.scatter(sample_pd["label"], sample_pd["prediction"], alpha=0.5)
    plt.xlabel("Valor Real")
    plt.ylabel("Valor Predicho")
    plt.title("Predicciones vs Valores Reales")

    # Línea ideal
    min_val = sample_pd["label"].min()
    max_val = sample_pd["label"].max()
    plt.plot([min_val, max_val], [min_val, max_val], 'r--')

    temp_plot_path = tempfile.mktemp(suffix=".png")
    plt.savefig(temp_plot_path)
    plt.close()

    mlflow.log_artifact(temp_plot_path)

    # ==============================
    # 3️⃣ Guardar modelo
    # ==============================

    mlflow.spark.log_model(model, "model")

    print(f"✓ RMSE: ${rmse:,.2f}")
    print("✓ Reporte y gráfico guardados en MLflow")



===  Modelo con Artefactos ===


2026/02/13 16:29:54 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmprfny_gmi/model, flavor: spark), fall back to return ['pyspark==3.5.0']. Set logging level to DEBUG to see the full traceback.


✓ RMSE: $1.39
✓ Reporte y gráfico guardados en MLflow


**Preguntas de Reflexión**

**1. ¿Qué ventajas tiene MLflow sobre guardar métricas en archivos CSV?**  
MLflow centraliza parámetros, métricas, modelos y artefactos en un mismo sistema versionado, permitiendo comparar experimentos fácilmente. Un CSV no mantiene trazabilidad, control de versiones ni almacenamiento de modelos asociados.

**2. ¿Cómo implementarías MLflow en un proyecto de equipo?**  
Se configuraría un tracking server centralizado accesible por todos los miembros. Cada experimento se registraría automáticamente desde los notebooks o pipelines, permitiendo auditoría, comparación y control de versiones colaborativo.

**3. ¿Qué artefactos adicionales guardarías además del modelo?**  
Gráficos de métricas, matrices de error, reportes de validación, pipelines completos de preprocesamiento, logs de entrenamiento, configuración del entorno y snapshots del dataset utilizado.

**4. ¿Cómo automatizarías el registro de experimentos?**  
Integrando MLflow dentro de pipelines CI/CD o jobs programados, donde cada entrenamiento registre automáticamente parámetros, métricas y artefactos sin intervención manual.


## **Notebook_11**

### **Configurar MLflow y MlflowClient**

In [8]:
from mlflow.tracking import MlflowClient

# Cliente del Model Registry
client = MlflowClient()

# Nombre del modelo en el Registry
model_name = "secop_prediccion_contratos"

print(f"MLflow URI: {mlflow.get_tracking_uri()}")
print(f"Modelo en Registry: {model_name}")


MLflow URI: http://mlflow:5000
Modelo en Registry: secop_prediccion_contratos


### **Entrenar y registrar modelo v1 (baseline)**

In [9]:
mlflow.set_experiment("SECOP_Model_Registry")

with mlflow.start_run(run_name="model_v1_baseline") as run:

    # Modelo SIN regularización
    lr = LinearRegression(
        featuresCol="features",
        labelCol="label",
        regParam=0.0,
        elasticNetParam=0.0,
        maxIter=100
    )

    model_v1 = lr.fit(train)

    predictions_v1 = model_v1.transform(test)
    rmse_v1 = evaluator_rmse.evaluate(predictions_v1)

    # Log
    mlflow.log_param("version", "1.0")
    mlflow.log_param("model_type", "baseline")
    mlflow.log_param("regParam", 0.0)
    mlflow.log_metric("rmse", rmse_v1)

    # Registro en el Model Registry
    mlflow.spark.log_model(
        spark_model=model_v1,
        artifact_path="model",
        registered_model_name=model_name
    )

    run_id_v1 = run.info.run_id
    print(f"Modelo v1 registrado | RMSE: ${rmse_v1:,.2f}")


2026/02/13 16:33:23 INFO mlflow.tracking.fluent: Experiment with name 'SECOP_Model_Registry' does not exist. Creating a new experiment.
26/02/13 16:33:24 WARN Instrumentation: [cd7e0882] regParam is zero, which might cause numerical instability and overfitting.
26/02/13 16:35:37 WARN NettyRpcEnv: Ignored failure: java.util.concurrent.TimeoutException: Cannot receive any reply from jupyter:35307 in 10000 milliseconds
2026/02/13 16:35:39 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpj5_biaok/model, flavor: spark), fall back to return ['pyspark==3.5.0']. Set logging level to DEBUG to see the full traceback.
Successfully registered model 'secop_prediccion_contratos'.
2026/02/13 16:35:42 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: secop_prediccion_contratos, version 1


Modelo v1 registrado | RMSE: $1.38


Created version '1' of model 'secop_prediccion_contratos'.


### **Entrenar y registrar modelo v2 (mejorado)**

¿Por qué versionar modelos en lugar de sobrescribir?

Porque permite trazabilidad, rollback seguro, auditoría y comparación histórica entre modelos. En producción nunca se debe perder el historial de versiones.

In [10]:
with mlflow.start_run(run_name="model_v2_regularized") as run:

    lr = LinearRegression(
        featuresCol="features",
        labelCol="label",
        regParam=0.1,
        elasticNetParam=0.5,
        maxIter=100
    )

    model_v2 = lr.fit(train)

    predictions_v2 = model_v2.transform(test)
    rmse_v2 = evaluator_rmse.evaluate(predictions_v2)

    mlflow.log_param("version", "2.0")
    mlflow.log_param("model_type", "regularized")
    mlflow.log_param("regParam", 0.1)
    mlflow.log_param("elasticNetParam", 0.5)
    mlflow.log_metric("rmse", rmse_v2)

    mlflow.spark.log_model(
        spark_model=model_v2,
        artifact_path="model",
        registered_model_name=model_name
    )

    print(f"Modelo v2 registrado | RMSE: ${rmse_v2:,.2f}")

2026/02/13 16:38:54 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpznm2pwuh/model, flavor: spark), fall back to return ['pyspark==3.5.0']. Set logging level to DEBUG to see the full traceback.
Registered model 'secop_prediccion_contratos' already exists. Creating a new version of this model...
2026/02/13 16:38:57 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: secop_prediccion_contratos, version 2


Modelo v2 registrado | RMSE: $1.42


Created version '2' of model 'secop_prediccion_contratos'.


In [11]:
print("\nComparación:")
print(f"v1 RMSE: ${rmse_v1:,.2f}")
print(f"v2 RMSE: ${rmse_v2:,.2f}")
print(f"Mejor modelo: {'v2' if rmse_v2 < rmse_v1 else 'v1'}")



Comparación:
v1 RMSE: $1.38
v2 RMSE: $1.42
Mejor modelo: v1


### **Gestionar stages: None → Staging → Production → Archived**

In [12]:
print("GESTIÓN DE VERSIONES EN MODEL REGISTRY")
print("="*60)

# Listar versiones registradas
model_versions = client.search_model_versions(f"name='{model_name}'")

print(f"\nVersiones actuales del modelo '{model_name}':")
for mv in model_versions:
    print(f"  - Versión {mv.version} | Stage: {mv.current_stage}")

# Determinar mejor versión según RMSE
print("\nComparando métricas:")
print(f"  v1 RMSE: ${rmse_v1:,.2f}")
print(f"  v2 RMSE: ${rmse_v2:,.2f}")

best_version = 2 if rmse_v2 < rmse_v1 else 1
worst_version = 1 if best_version == 2 else 2

print(f"\nMejor versión: v{best_version}")

# Promover mejor versión a Staging
client.transition_model_version_stage(
    name=model_name,
    version=best_version,
    stage="Staging"
)

print(f"v{best_version} -> Staging")

# Simular validación (aquí ya se sabe cuál es mejor)
client.transition_model_version_stage(
    name=model_name,
    version=best_version,
    stage="Production"
)

print(f"v{best_version} -> Production")

# Archivar versión anterior
client.transition_model_version_stage(
    name=model_name,
    version=worst_version,
    stage="Archived"
)

print(f"v{worst_version} -> Archived")

# Verificar estado final
print("\nEstado final de versiones:")
model_versions = client.search_model_versions(f"name='{model_name}'")
for mv in model_versions:
    print(f"  - Versión {mv.version} | Stage: {mv.current_stage}")



GESTIÓN DE VERSIONES EN MODEL REGISTRY

Versiones actuales del modelo 'secop_prediccion_contratos':
  - Versión 2 | Stage: None
  - Versión 1 | Stage: None

Comparando métricas:
  v1 RMSE: $1.38
  v2 RMSE: $1.42

Mejor versión: v1


/tmp/ipykernel_4387/2731657312.py:22: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.9.2/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


v1 -> Staging


/tmp/ipykernel_4387/2731657312.py:31: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.9.2/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


v1 -> Production


/tmp/ipykernel_4387/2731657312.py:40: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.9.2/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


v2 -> Archived

Estado final de versiones:
  - Versión 2 | Stage: Archived
  - Versión 1 | Stage: Production


### **Agregar metadata y descripcion al modelo**

**¿Qué información mínima debería tener cada versión?**

Cada versión de modelo debe incluir como mínimo:
- Métrica principal de validación (RMSE u otra relevante)
- Fecha de entrenamiento
- Dataset utilizado
- Tipo de modelo y configuración principal
- Responsable o autor
- Estado del modelo (Staging o Production)

Esto garantiza trazabilidad, auditoría y reproducibilidad en entornos productivos.


In [13]:
# Identificar versión en Production
from mlflow.tracking import MlflowClient

client = MlflowClient()

print("\nBuscando versión en Production...")

production_version = None

for mv in client.search_model_versions(f"name='{model_name}'"):
    if mv.current_stage == "Production":
        production_version = mv.version
        break

print(f"Versión en Production: {production_version}")



Buscando versión en Production...
Versión en Production: 1


In [14]:
# Agregar Descripción Profesional
client.update_model_version(
    name=model_name,
    version=production_version,
    description=f"""
Modelo oficial en producción para predicción de valor de contratos SECOP.

• Versión: {production_version}
• RMSE validación: ${rmse_v1:,.2f}
• Dataset: secop_ml_ready.parquet
• Features: PCA
• Autor: Diego_Gomez_and_Victor_Diaz
• Fecha: 2026-02-13

Modelo seleccionado tras comparación entre baseline y modelo regularizado.
"""
)

print(f"✓ Metadata agregada a versión {production_version}")


✓ Metadata agregada a versión 1


In [15]:
# Agregar Tags 
client.set_model_version_tag(
    name=model_name,
    version=production_version,
    key="status",
    value="validated"
)

client.set_model_version_tag(
    name=model_name,
    version=production_version,
    key="area",
    value="finanzas_publicas"
)

client.set_model_version_tag(
    name=model_name,
    version=production_version,
    key="framework",
    value="SparkML"
)

print("✓ Tags agregados correctamente")


✓ Tags agregados correctamente


### **Cargar modelo desde Registry para prediccion**

In [16]:
print("CARGANDO MODELO DESDE PRODUCTION")
print("="*60)

# Definir URI usando nombre y stage (NO ruta de archivo)
model_uri = f"models:/{model_name}/Production"

# Cargar modelo desde Registry
loaded_model = mlflow.spark.load_model(model_uri)

print(f"Modelo cargado desde: {model_uri}")
print(f"Tipo de objeto: {type(loaded_model)}")

# Verificar que funciona haciendo predicciones
test_predictions = loaded_model.transform(test)

# Evaluar nuevamente
test_rmse = evaluator_rmse.evaluate(test_predictions)

print(f"\nRMSE verificación: ${test_rmse:,.2f}")
print("="*60)


CARGANDO MODELO DESDE PRODUCTION


/usr/local/lib/python3.11/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.9.2/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])
2026/02/13 16:42:40 INFO mlflow.spark: 'models:/secop_prediccion_contratos/Production' resolved as 'file:///opt/mlflow/mlruns/315159979831279198/8f40e77831fd416594f67b33b71a1019/artifacts/model'
2026/02/13 16:42:41 INFO mlflow.spark: URI 'models:/secop_prediccion_contratos/Production/sparkml' does not point to the current DFS.
2026/02/13 16:42:41 INFO mlflow.spark: File 'models:/secop_prediccion_contratos/Production/sparkml' not found on DFS. Will attempt to upload the file.


Modelo cargado desde: models:/secop_prediccion_contratos/Production
Tipo de objeto: <class 'pyspark.ml.pipeline.PipelineModel'>



RMSE verificación: $1.38


1. ¿Cómo harías rollback si el modelo en Production falla?

Si el modelo en Production presenta fallos, realizaría rollback promoviendo la versión anterior desde "Archived" o "Staging" nuevamente a "Production" utilizando transition_model_version_stage(). Esto permite revertir el modelo activo sin modificar el código de producción, ya que el sistema siempre carga el modelo por nombre y stage.

2. ¿Qué criterios usarías para promover un modelo de Staging a Production?

Promovería un modelo a Production únicamente si:

  - Presenta mejor RMSE/MAE/R² que la versión actual.
  - Supera validaciones técnicas y de negocio.
  - Es estable en pruebas controladas.
  - No introduce sesgos o comportamientos inesperados.
  - Ha sido revisado y aprobado por el equipo responsable.

3. ¿Cómo implementarías A/B testing con el Model Registry?

Implementaría A/B testing desplegando dos versiones diferentes del modelo (por ejemplo, Production y Staging) y enviando un porcentaje del tráfico a cada uno. Posteriormente compararía métricas en producción (errores reales, impacto financiero, latencia) para determinar cuál modelo ofrece mejor rendimiento antes de hacer la promoción definitiva.

4. ¿Quién debería tener permisos para promover modelos a Production?

Solo deberían tener permisos para promover modelos a Production los roles responsables de MLOps o Data Science Lead, ya que este proceso impacta directamente el entorno productivo. Esto evita errores humanos y mantiene control y trazabilidad sobre cambios críticos.

## **Notebook_12**

1. ¿Por qué cargar desde el Registry en lugar de una ruta de archivo? ¿Qué ventajas tiene para un sistema de producción?

  - Permite cambiar la versión en Production sin modificar código.
  - Facilita rollback inmediato.
  - Centraliza control y gobernanza.
  - Garantiza trazabilidad y versionamiento.

2. ¿Qué pasaría si no hay modelo en Production?

El sistema lanzaría un error porque no existe una versión promovida a ese stage.

3. ¿Cómo manejar ese error?

  - Capturar la excepción (como hicimos arriba).
  - Enviar alerta.
  - Cargar versión anterior estable.
  - Detener el pipeline si es crítico.

### **Cargar modelo en Production desde MLflow Registry**

In [17]:
mlflow.set_tracking_uri("http://mlflow:5000")
# nombre del modelo registrado
model_name = "secop_prediccion_contratos"
model_uri = f"models:/{model_name}/Production"

print("CARGANDO MODELO DESDE REGISTRY")
print("="*60)
print(f"URI: {model_uri}")

try:
    production_model = mlflow.spark.load_model(model_uri)
    print("✓ Modelo cargado correctamente")
    print(f"Tipo: {type(production_model)}")

except Exception as e:
    print("No existe modelo en Production")
    print("Error:", e)


CARGANDO MODELO DESDE REGISTRY
URI: models:/secop_prediccion_contratos/Production


2026/02/13 16:43:25 INFO mlflow.spark: 'models:/secop_prediccion_contratos/Production' resolved as 'file:///opt/mlflow/mlruns/315159979831279198/8f40e77831fd416594f67b33b71a1019/artifacts/model'
2026/02/13 16:43:26 INFO mlflow.spark: URI 'models:/secop_prediccion_contratos/Production/sparkml' does not point to the current DFS.
2026/02/13 16:43:26 INFO mlflow.spark: File 'models:/secop_prediccion_contratos/Production/sparkml' not found on DFS. Will attempt to upload the file.


✓ Modelo cargado correctamente
Tipo: <class 'pyspark.ml.pipeline.PipelineModel'>


### **Preparar datos nuevos para prediccion**

In [18]:
df_new = spark.read.parquet("/opt/spark-data/raw/secop_ml_ready1.parquet")
df_new = df_new.withColumnRenamed("features_pca", "features")

# Simular que no tenemos label en producción
df_new_no_label = df_new.drop("valor_del_contrato_log")

print(f"Registros para predicción: {df_new_no_label.count():,}")
print("Columnas disponibles:")
print(df_new_no_label.columns)

Registros para predicción: 100,000
Columnas disponibles:
['features']


### **Generar predicciones batch con timestamp**

In [19]:
from pyspark.sql.functions import current_timestamp

predictions_batch = production_model.transform(df_new_no_label)

predictions_batch = predictions_batch.withColumn(
    "prediction_timestamp",
    current_timestamp()
)

predictions_batch.select(
    "features",
    "prediction",
    "prediction_timestamp"
).show(10, truncate=False)



+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------+--------------------------+
|features                                                                                                                                                                                                                                                                                                                                                              

¿Por qué agregar timestamp?

 - Auditoría
 - Trazabilidad
 - Comparación entre lotes
 - Monitoreo temporal

### **Monitorear predicciones (estadisticas, anomalias, rangos)**

¿Cómo detectar data drift?

  - Comparar media y desviación vs entrenamiento
  - Monitorear distribución por rangos
  - Alertas si cambia > X%
  - Validar features entrantes

Si hay drift, toca tener alertas, Evaluar retraining y Revertir versión

In [20]:
from pyspark.sql.functions import min as spark_min, max as spark_max, avg, stddev, count
from pyspark.sql.functions import exp

predictions_batch = predictions_batch.withColumn(
    "prediction_real",
    exp(col("prediction"))
)

stats = predictions_batch.select(
    spark_min("prediction_real").alias("min_pred"),
    spark_max("prediction_real").alias("max_pred"),
    avg("prediction_real").alias("avg_pred"),
    stddev("prediction_real").alias("std_pred"),
    count("*").alias("total")
).collect()[0]


print("=== ESTADÍSTICAS DE PREDICCIONES ===")
print(f"Total: {stats['total']:,}")
print(f"Mínimo: ${stats['min_pred']:,.2f}")
print(f"Máximo: ${stats['max_pred']:,.2f}")
print(f"Promedio: ${stats['avg_pred']:,.2f}")
print(f"Std: ${stats['std_pred']:,.2f}")

from pyspark.sql.functions import when

prediction_ranges = predictions_batch.select(
    count(when(col("prediction_real") < 10000000, True)).alias("< 10M"),
    count(when((col("prediction_real") >= 10000000) & (col("prediction_real") < 100000000), True)).alias("10M-100M"),
    count(when((col("prediction_real") >= 100000000) & (col("prediction_real") < 1000000000), True)).alias("100M-1B"),
    count(when(col("prediction_real") >= 1000000000, True)).alias("> 1B")
)


print("=== DISTRIBUCIÓN POR RANGOS ===")
prediction_ranges.show()


=== ESTADÍSTICAS DE PREDICCIONES ===
Total: 100,000
Mínimo: $0.00
Máximo: $784,501,078,948.27
Promedio: $113,620,498.46
Std: $3,798,692,767.67
=== DISTRIBUCIÓN POR RANGOS ===
+-----+--------+-------+----+
|< 10M|10M-100M|100M-1B|> 1B|
+-----+--------+-------+----+
|52776|   37414|   8759|1051|
+-----+--------+-------+----+



### **Dectectar anomalias**

In [21]:
anomalias = predictions_batch.filter(col("prediction_real") < 0).count()
print(f"Predicciones negativas: {anomalias}")

Predicciones negativas: 0


In [7]:
### Cargar modelos registrados ####

import mlflow.spark
from pyspark.sql import SparkSession

# 1. Definir la ruta usando el nombre que elegiste
# 'models:/' le dice a MLflow que busque en el registro oficial, no en una carpeta
model_name = "elastic"
model_version = "latest" # O puedes poner "1", "2", etc.
model_uri = f"models:/{model_name}/{model_version}"

# 2. Cargar el modelo como un objeto de Spark ML
print(f"Cargando modelo '{model_name}' desde el registro...")
loaded_model = mlflow.spark.load_model(model_uri)

print("✓ Modelo cargado exitosamente")

2026/01/30 01:48:07 INFO mlflow.spark: 'models:/elastic/latest' resolved as 'file:///opt/mlflow/mlruns/402490040680732574/42e2221c257d4694a5621b582fd62aa1/artifacts/model'


Cargando modelo 'elastic' desde el registro...


2026/01/30 01:48:07 INFO mlflow.spark: URI 'models:/elastic/latest/sparkml' does not point to the current DFS.
2026/01/30 01:48:07 INFO mlflow.spark: File 'models:/elastic/latest/sparkml' not found on DFS. Will attempt to upload the file.


✓ Modelo cargado exitosamente


In [8]:
# 3. Supongamos que 'df_nuevos_contratos' son datos que acaban de llegar
# (Deben haber pasado por el mismo Pipeline de Scaler y PCA)
predictions = loaded_model.transform(df)

# 4. Mostrar los resultados
predictions.select("features", "prediction").show(5)

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|[1.05511824914886...|1.579372016765363E11|
|[2.89965949085992...|-9.83453525732329E10|
|[1.91826045058565...|  8.51104667218737E9|
|[1.91000751697583...|3.685598427820685...|
|[1.91815735697773...|  8.53426230649346E9|
+--------------------+--------------------+
only showing top 5 rows



In [9]:
spark.stop()

In [18]:
import mlflow
from mlflow.tracking import MlflowClient

# 1. Asegúrate de configurar la URI primero
mlflow.set_tracking_uri("http://mlflow:5000")
client = MlflowClient()

# 2. Listar experimentos para verificar el nombre real
print("Experimentos disponibles en el servidor:")
for exp in client.search_experiments():
    print(f" - {exp.name}")

# 3. Intentar obtener el experimento con el nombre correcto
experiment_name = "secop_prediccion" # Verifica si coincide con la lista de arriba
experiment = client.get_experiment_by_name(experiment_name)

if experiment is None:
    raise ValueError(f"No se encontró el experimento '{experiment_name}'. Revisa la lista de arriba.")

# 4. Si existe, buscar los runs
runs = client.search_runs(
    experiment_ids=[experiment.experiment_id],
    order_by=["metrics.rmse ASC"],
    max_results=1
)

if not runs:
    raise ValueError(f"El experimento '{experiment_name}' existe pero no tiene ninguna ejecución (runs).")

best_run = runs[0]
best_run_id = best_run.info.run_id
print(f"✓ Éxito. Mejor Run ID: {best_run_id}")

Experimentos disponibles en el servidor:
 - secop_prediccion
✓ Éxito. Mejor Run ID: 42e2221c257d4694a5621b582fd62aa1


In [19]:
import mlflow

# 1. Configuración de acceso (asegúrate de que la URI sea la correcta)
mlflow.set_tracking_uri("http://mlflow:5000")

# 2. Ruta del modelo usando el ID que ya identificamos
# (Asegúrate de que 'best_run_id' esté definido en tu sesión actual)
model_uri = f"runs:/{best_run_id}/model"

# 3. Registrar con el nombre 'mejor'
# Si ya existía uno llamado 'mejor', creará la Versión 2, 3, etc.
model_details = mlflow.register_model(model_uri, "mejor")

print("-" * 30)
print(f"MODELO REGISTRADO COMO: mejor")
print("-" * 30)
print(f"Versión: {model_details.version}")
print(f"Estado: {model_details.current_stage}")

Successfully registered model 'mejor'.
2026/01/30 01:54:29 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: mejor, version 1


------------------------------
MODELO REGISTRADO COMO: mejor
------------------------------
Versión: 1
Estado: None


Created version '1' of model 'mejor'.
